In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation
# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징을 두드러지게 한다.
from tensorflow.keras.layers import Conv2D
# MaxPooling : 불필요한 부분을 제거하는 역할을 한다
from tensorflow.keras.layers import MaxPool2D
# Flatten : CNN, MaxPooling은 2차원 데이터를 사용하고 출력하기 때문에
# 1차원 데이터로 변하는 것을 사용해야 한다.
from tensorflow.keras.layers import Flatten
# Dropout : 노드의 일부를 on/off 하면서 과적합을 예방한다.
from tensorflow.keras.layers import Dropout
# Embedding : 주어진 단어 데이터를 원핫 인코딩하고 차원축소를 한다.
from tensorflow.keras.layers import Embedding
# 문장 학습을 위한 RNN(LSTM)
from tensorflow.keras.layers import LSTM
# Cnn1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D


# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# 문자열을 숫자로 변환
from sklearn.preprocessing import LabelEncoder
# 원핫 인코딩
from tensorflow.keras.utils import to_categorical

# 학습데이터와 검증데이터로 나눈다
from sklearn.model_selection import train_test_split

# K폴드 교차검증
from sklearn.model_selection import KFold

# 저장된 모델을 복원한다.
from tensorflow.keras.models import load_model

# epoch 마다 저장을 하기 위해 사용한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 조기 중단을 위해 사용한다.
from tensorflow.keras.callbacks import EarlyStopping

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장의 데이터의 단어 데이터 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 단어사전으로 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import os
import glob
from PIL import Image

# VGG16
from tensorflow.keras.applications import VGG16

# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

### 전이학습
- 기존에 학습이 완료된 모델을 이용하는 것을 의미.
- 여기에서는 천만장이 넘는 이미지를 학습시킨 모델을 사용한다.

In [2]:
test_model = VGG16(include_top=False, input_shape=(64, 64, 3), 
                  weights='imagenet')
test_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [3]:
# seed 설정
np.random.seed(1)
tf.random.set_seed(1)

In [4]:
# 폴더 목록을 가져온다.
root_dir = '101_ObjectCategories'

a1 = os.walk(root_dir)
categories = list(a1)[0][1]

# 수업 실습을 위해...(나중에 주석처리해서 테스트 해보세요)
categories = categories[:5]

categories


['accordion', 'airplanes', 'anchor', 'ant', 'BACKGROUND_Google']

In [5]:
# 이미지를 복원한다.
X_train, X_test, y_train, y_test = np.load('101_new2.npy',
                                          allow_pickle=True)


In [6]:
# 데이터 정규화(0 ~ 1 사이로.)
X_train = X_train.astype('float') / 255
X_test = X_test.astype('float') / 255

print(X_train.shape)
print(X_test.shape)


(17926, 64, 64, 3)
(5976, 64, 64, 3)


In [7]:
# 결과 데이터 원핫 인코딩
y_train = to_categorical(y_train, len(categories))
y_test = to_categorical(y_test, len(categories))


In [8]:
# VGG 16 모델 사용
vgg16_model = VGG16(include_top=False, input_shape=(64, 64, 3),
                   weights='imagenet')
# VGG16모델은 학습을 하지 않도록 설정한다.
# 이미 학습이 완료된거기 때문에 학습시키면 안됨
vgg16_model.trainable = False

# 모델 구축하기
model = Sequential()

# 모델 구축하기
model = Sequential()

# VGG16모델을 연결한다.
model.add(vgg16_model)

model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(len(categories)))
model.add(Activation('softmax'))



model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 activation (Activation)     (None, 4096)              0         
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 activation_1 (Activation)   (None, 4096)             

In [9]:
# 모델들이 저장될 파일명
# epoch : 학습 횟수
# val_loss : 검증 데이터 손실률
path1 = 'model/30/{epoch}-{val_loss}.h5'
path2 = 'model/30/최종.h5'

# 저장 콜백 설정
# save_best_only : True로 설정하면 개선이 되었을 때만 저장한다.
call1 = ModelCheckpoint(filepath=path1, monitor='val_loss', 
                        save_best_only=True)
call2 = ModelCheckpoint(filepath=path2, monitor='val_loss',
                        save_best_only=True)

# 조기 중단 콜백
# patience : 개선이 된 이후 몇 번을 더 볼 것인가를 설정한다.
# 이 횟수 동안 개선이 되지 않으면 중단 시킨다.
call3 = EarlyStopping(monitor='val_loss', patience=50)


In [10]:
# 학습 모델을 생성한다.(다중분류)
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])


In [11]:
# 학습한다.
with tf.device('/CPU:0') :
    history = model.fit(X_train, y_train, epochs=10, batch_size=500, 
          validation_data=[X_test, y_test],
          callbacks=[call1, call2, call3])

Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 1.1367 - accuracy: 0.7880

KeyboardInterrupt: 

In [ ]:
# 모델을 복원한다.
model2 = load_model(path2)
model2


In [ ]:
with tf.device('/CPU:0') :
    a1 = model2.evaluate(X_test, y_test)
    print(f'손실률 : {a1[0]}')
    print(f'정확도 : {a1[1]}')


In [ ]:
# 오차 정도와 정확도를 그래프로 그린다.
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
